### Predicts 10 blogs out of test data which are ging to receive most number of comments.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Load training data

In [2]:
df = pd.read_csv('/Users/gauravbishnoi/datas/blog_comments/blogData_train.csv', header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,271,272,273,274,275,276,277,278,279,280
0,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.1, random_state=42)

In [6]:
def try_ols(xtrain, xtest, ytrain, ytest):
    from sklearn.linear_model import LinearRegression
    ols = LinearRegression()
    ols.fit(xtrain, ytrain)
    ols_predictions = ols.predict(xtest)
    top50_pred = ols_predictions.argsort()[-50:][::-1]
    top50_test = ytest.values.argsort()[-50:][::-1]
    print("top 10 score: {}".format(sum(np.in1d(top50_pred[:10],top50_test[:10]))/10))
    print("top 20 score: {}".format(sum(np.in1d(top50_pred[:20],top50_test[:20]))/20))
    print("top 50 score: {}".format(sum(np.in1d(top50_pred,top50_test))/50))

In [7]:
try_ols(x_train, x_test, y_train, y_test)

top 10 score: 0.2
top 20 score: 0.35
top 50 score: 0.48


Removing bag of words features.

In [8]:
xtrain2, xtest2, ytrain2, ytest2 = train_test_split(df.iloc[:,:61], df.iloc[:,-1], test_size=0.1, random_state=42)

In [9]:
try_ols(xtrain2, xtest2, ytrain2, ytest2)

top 10 score: 0.2
top 20 score: 0.35
top 50 score: 0.48


There is no effect of bag of words features on output

Try Ridge Regression, it may perform better than ols because of collinearity among features (as shown in heatmap in 'blog_analysis_quick.ipynb').

In [10]:
def try_ridge(xtrain, xtest, ytrain, ytest):
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import GridSearchCV
    alps = [0.01, 0.1, 1, 10]
    ridgegrid_param = {'alpha': alps}
    ridge_grid = GridSearchCV(estimator=Ridge(), param_grid=ridgegrid_param, verbose=3)
    ridge_grid.fit(xtrain, ytrain)
    alp = ridge_grid.best_estimator_.alpha
    
    ridge = Ridge(alpha=alp)
    ridge.fit(xtrain, ytrain)
    ridge_predictions = ridge.predict(xtest)
    ridge50_pred = ridge_predictions.argsort()[-50:][::-1]
    ridge50_test = ytest.values.argsort()[-50:][::-1]
    print("\n\n")
    print("top 10 score: {}".format(sum(np.in1d(ridge50_pred[:10],ridge50_test[:10]))/10))
    print("top 20 score: {}".format(sum(np.in1d(ridge50_pred[:20],ridge50_test[:20]))/20))
    print("top 50 score: {}".format(sum(np.in1d(ridge50_pred,ridge50_test))/50))

In [11]:
try_ridge(xtrain2, xtest2, ytrain2, ytest2)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] alpha=0.01 ......................................................
[CV] ....................... alpha=0.01, score=0.401313, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ....................... alpha=0.01, score=0.349296, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ....................... alpha=0.01, score=0.351982, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................ alpha=0.1, score=0.401317, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................ alpha=0.1, score=0.349300, total=   0.0s
[CV] alpha=0.1 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ........................ alpha=0.1, score=0.351993, total=   0.0s
[CV] alpha=1 .........................................................
[CV] .......................... alpha=1, score=0.401355, total=   0.0s
[CV] alpha=1 .........................................................
[CV] .......................... alpha=1, score=0.349331, total=   0.0s
[CV] alpha=1 .........................................................
[CV] .......................... alpha=1, score=0.352048, total=   0.0s
[CV] alpha=10 ........................................................
[CV] ......................... alpha=10, score=0.401501, total=   0.0s
[CV] alpha=10 ........................................................
[CV] ......................... alpha=10, score=0.349422, total=   0.0s
[CV] alpha=10 ........................................................
[CV] ......................... alpha=10, score=0.352114, total=   0.0s



top 10 score: 0.2
top 20 score: 0.35
top 50 score: 0.48


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.5s finished


Ridge regression is giving similar results to OLS

In [12]:
def try_dtree(xtrain, xtest, ytrain, ytest):
    from sklearn.tree import DecisionTreeRegressor
    dtree = DecisionTreeRegressor()
    dtree.fit(xtrain, ytrain)
    dtree_predictions = dtree.predict(xtest)
    dtree50_pred = dtree_predictions.argsort()[-50:][::-1]
    dtree50_test = ytest.argsort()[-50:][::-1]
    print("top 10 score: {}".format(sum(np.in1d(dtree50_pred[:10],dtree50_test[:10]))/10))
    print("top 20 score: {}".format(sum(np.in1d(dtree50_pred[:20],dtree50_test[:20]))/20))
    print("top 50 score: {}".format(sum(np.in1d(dtree50_pred,dtree50_test))/50))

In [13]:
try_dtree(xtrain2, xtest2, ytrain2, ytest2)

top 10 score: 0.4
top 20 score: 0.35
top 50 score: 0.5


In [14]:
try_dtree(x_train, x_test, y_train, y_test)

top 10 score: 0.2
top 20 score: 0.5
top 50 score: 0.44


In [28]:
def try_rfr(xtrain, xtest, ytrain, ytest):
    from sklearn.ensemble import RandomForestRegressor
    rfr = RandomForestRegressor(n_estimators=20)
    rfr.fit(xtrain, ytrain)
    rfr_predictions = rfr.predict(xtest)
    rfr50_pred = rfr_predictions.argsort()[-50:][::-1]
    rfr50_test = ytest.argsort()[-50:][::-1]
    print("top 10 score: {}".format(sum(np.in1d(rfr50_pred[:10],rfr50_test[:10]))/10))
    print("top 20 score: {}".format(sum(np.in1d(rfr50_pred[:20],rfr50_test[:20]))/20))
    print("top 50 score: {}".format(sum(np.in1d(rfr50_pred,rfr50_test))/50))

In [29]:
try_rfr(xtrain2, xtest2, ytrain2, ytest2)

top 10 score: 0.4
top 20 score: 0.45
top 50 score: 0.44


In [21]:
from sklearn.ensemble import RandomForestRegressor

In [31]:
rfr = RandomForestRegressor(n_estimators=25)

In [ ]:
rfr.fit(xtrain2, ytrain2)

In [30]:
rfr_predictions = rfr.predict(xtest2)
rfr50_pred = rfr_predictions.argsort()[-50:][::-1]
rfr50_test = ytest2.argsort()[-50:][::-1]
print("top 10 score: {}".format(sum(np.in1d(rfr50_pred[:10],rfr50_test[:10]))/10))
print("top 20 score: {}".format(sum(np.in1d(rfr50_pred[:20],rfr50_test[:20]))/20))
print("top 50 score: {}".format(sum(np.in1d(rfr50_pred,rfr50_test))/50))

top 10 score: 0.4
top 20 score: 0.55
top 50 score: 0.52


In [100]:
'''
dtree_scaler = StandardScaler()
xtrain_dtree = dtree_scaler.fit_transform(x_train)
xtest_dtree = dtree_scaler.transform(x_test)
dtree_scaler_y = StandardScaler()
ytrain_dtree = dtree_scaler_y.fit_transform(ytrain2.values.reshape(-1,1))
ytest_dtree = dtree_scaler_y.transform(ytest2.values.reshape(-1,1))
'''

'\ndtree_scaler = StandardScaler()\nxtrain_dtree = dtree_scaler.fit_transform(x_train)\nxtest_dtree = dtree_scaler.transform(x_test)\ndtree_scaler_y = StandardScaler()\nytrain_dtree = dtree_scaler_y.fit_transform(ytrain2.values.reshape(-1,1))\nytest_dtree = dtree_scaler_y.transform(ytest2.values.reshape(-1,1))\n'